In [30]:
import cv2
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from itertools import combinations

In [31]:
CAMERA_WALL_DISTANCE = 1 # m

data_dir = Path("data")

devices = {}

for device_dir in data_dir.iterdir():
	if not device_dir.is_dir():
		continue
	print(device_dir.name)

	images = {}
	for image_dir in device_dir.glob("*.png"):
		images[image_dir.stem] = cv2.imread(str(image_dir))

	devices[device_dir.name] = images



1844301001298E0E00_5_1
1844301001298E0E00_5_2
18443010014D8E0E00_52
184430100155960F00_50
1844301001BB8E0E00_45
184430101138970F00_36
1844301011E0960F00_46
1844301011E59B0F00_41
184430102111970F00_17
18443010212C8E0E00_48
1844301021398E0E00_27
1844301021E0960F00_2
184430103138970F00_33
184430103154970F00_44
184430103164970F00_32
184430104138970F00_26
184430104169980F00_40
1844301041B09A0F00_35
184430105114970F00_7
18443010512F8E0E00_42
184430105164970F00_24
1844301051969A0F00_43
1844301051F19A0F00_49
18443010616C980F00_16
1844301061899A0F00_20
1844301061D19B0F00_1
1844301061F0960F00_14_1
1844301061F0960F00_14_2
184430107157970F00_15
1844301071C09A0F00_37
18443010812F8E0E00_19
18443010813B970F00_10
1844301081899A0F00_9_1
1844301081899A0F00_9_2
1844301081AA8E0E00_18
1844301081E3960F00_3
1844301081F0960F00_22
184430109124970F00_25
1844301091C39A0F00_11
18443010A1C39A0F00_6
18443010B13F8E0E00_31
18443010B1C39A0F00_38
18443010C1328E0E00_34
18443010C1D39A0F00_39
18443010C1D49B0F00_28
1844301

In [32]:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

def get_corners(image):
	""" Get the corners of the noise pattern board """
	
	corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, aruco_dict)
	inds = np.argsort(ids.flatten())
	corners_sorted = np.array(corners)[inds]
	if len(corners_sorted) != 4:
		raise Exception("Noise pattern board not found")

	center = np.array([m.mean(axis=1).flatten() for m in corners_sorted]).mean(axis=0)

	points = []
	for m in corners_sorted:
		i = np.linalg.norm(m - center, axis=-1).argmin()
		points.append(m[:, i])

	points = np.array(points).reshape(-1, 2)

	return points

def get_center(points):
	center = points.mean(axis=0)
	return center[0], center[1]

def get_rotation(points):
	center_bottom = (points[0] + points[1]) / 2
	center_top = (points[2] + points[3]) / 2
	d = center_bottom - center_top
	angle = np.arctan2(*d)
	return angle

def get_board_width(points):
	return np.linalg.norm(points[0] - points[1])
	

In [33]:
center_bottom = np.array([-4, 600])
center_top = np.array([0, 0])
d = center_bottom - center_top
angle = np.arctan2(*d)
angle_deg = np.rad2deg(angle)
print(angle_deg)

-0.3819662047290255


In [45]:
results = []

offsets = { # compensation for different camera positions (in m)
    "right": -0.0375,
    "rectified_right": -0.0375,
    "color": 0,
    "left": 0.0375,
    "rectified_left": 0.0375,
}

for mxid, images in devices.items():
    result = {"mxid": mxid}

    for name, img in images.items():
        points = get_corners(img)
        center_x, center_y = get_center(points)
        angle = get_rotation(points)
        board_width = get_board_width(points)
        px2m = 0.6 / board_width

        offset = offsets[name]

        result[f"{name}_roll_angle"] = np.rad2deg(angle)
        result[f"{name}_yaw_angle"] = np.rad2deg(np.arctan2(offset + px2m * (img.shape[1]/2 - center_x), CAMERA_WALL_DISTANCE))
        result[f"{name}_pitch_angle"] = np.rad2deg(np.arctan2(px2m * (img.shape[0]/2 - center_y), CAMERA_WALL_DISTANCE))


    for cam_a, cam_b in combinations(["left", "right", "color"], 2):
        result[f"{cam_a}_{cam_b}_roll_diff"] = np.abs(result[f"{cam_a}_roll_angle"] - result[f"{cam_b}_roll_angle"])
        result[f"{cam_a}_{cam_b}_yaw_diff"] = np.abs(result[f"{cam_a}_yaw_angle"] - result[f"{cam_b}_yaw_angle"])
        result[f"{cam_a}_{cam_b}_pitch_diff"] = np.abs(result[f"{cam_a}_pitch_angle"] - result[f"{cam_b}_pitch_angle"])

    for cam_a, cam_b in combinations(["rectified_left", "rectified_right"], 2):
        result[f"{cam_a}_{cam_b}_roll_diff"] = np.abs(result[f"{cam_a}_roll_angle"] - result[f"{cam_b}_roll_angle"])
        result[f"{cam_a}_{cam_b}_yaw_diff"] = np.abs(result[f"{cam_a}_yaw_angle"] - result[f"{cam_b}_yaw_angle"])
        result[f"{cam_a}_{cam_b}_pitch_diff"] = np.abs(result[f"{cam_a}_pitch_angle"] - result[f"{cam_b}_pitch_angle"])

    results.append(result)

results_df = pd.DataFrame(results)
results_df.transpose()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
mxid,1844301001298E0E00_5_1,1844301001298E0E00_5_2,18443010014D8E0E00_52,184430100155960F00_50,1844301001BB8E0E00_45,184430101138970F00_36,1844301011E0960F00_46,1844301011E59B0F00_41,184430102111970F00_17,18443010212C8E0E00_48,...,18443010E1D39A0F00_21,18443010E1E6960F00_29,18443010F1358E0E00_8_1,18443010F1358E0E00_8_2,18443010F1C99A0F00_47,18443010F1DC960F00_23,18443010F1E19B0F00_4_1,18443010F1E19B0F00_4_2,18443010F1E68E0E00_13,18443010F1E9960F00_12
color_roll_angle,-0.093468,0.046964,0.0,-0.705577,0.470398,0.468475,-0.613018,-0.188008,-0.140776,-0.422666,...,-0.749534,0.046964,-0.046925,0.046772,-0.094469,-0.563085,0.0,0.0,0.420598,-0.28063
color_yaw_angle,1.340404,0.632117,0.042441,-0.958655,0.681266,-0.493914,-1.468559,0.637626,1.213371,0.162813,...,1.513183,0.925029,0.219097,-0.99399,-0.342338,1.077632,0.834616,-0.888248,1.087297,0.633968
color_pitch_angle,1.39682,1.377662,-1.838477,0.553924,0.624499,1.234625,-0.305127,0.998887,0.589014,-0.042473,...,-0.007073,0.94621,-0.826861,-0.853021,-0.206831,1.382383,0.629517,0.648586,0.491084,-1.302984
left_roll_angle,-0.120369,0.0,0.120877,0.120623,0.240737,-0.23973,-0.120369,-0.360346,0.120369,-0.240737,...,-0.962864,0.0,-0.844298,-0.842524,-0.360346,-0.240232,-0.241244,-0.481466,0.120117,0.120117
left_yaw_angle,0.067924,-0.637711,-0.704023,-0.144362,-0.848535,-0.517896,-0.655758,0.900172,-0.692033,-2.264774,...,0.704443,-1.071881,2.52766,1.334293,0.788783,-0.239604,2.111221,0.408093,-0.516712,0.977367
left_pitch_angle,0.0,-0.018093,0.254375,-0.234711,-1.191482,0.900771,-0.723672,-0.054276,1.266329,-1.700187,...,-0.28882,0.66942,-0.235676,-0.253261,0.380727,0.705568,-0.637232,-0.652656,0.960875,0.090087
rectified_left_roll_angle,0.0,0.120117,0.121133,0.361863,0.722177,0.119865,0.481466,0.119865,0.0,0.241244,...,0.119865,-0.120369,0.480456,0.722177,0.0,0.240737,0.120623,0.120369,-0.119865,0.240232
rectified_left_yaw_angle,0.954349,0.248787,0.637281,1.657542,0.194573,-0.175665,0.085948,1.984957,-0.099718,-1.368666,...,1.695762,0.140241,2.220006,0.990525,0.69806,0.483987,2.456628,0.749479,-0.14925,2.183645
rectified_left_pitch_angle,-0.05428,-0.090467,0.018208,-0.090849,-1.013089,0.936827,-0.832236,0.10856,1.395852,-1.577045,...,-0.289491,0.651329,-0.670836,-0.651329,0.108787,0.796059,-0.382371,-0.36339,0.759872,0.234719


In [37]:
results_df[["right_color_roll_diff", "right_color_yaw_diff", "right_color_pitch_diff"]].mean()

right_color_roll_diff     0.307865
right_color_yaw_diff      1.228734
right_color_pitch_diff    0.961858
dtype: float64

In [38]:
results_df.std(numeric_only=True)

color_roll_angle                             0.314517
color_yaw_angle                              0.771358
color_pitch_angle                            0.758360
left_roll_angle                              0.273383
left_yaw_angle                               1.214164
left_pitch_angle                             0.934087
rectified_left_roll_angle                    0.190180
rectified_left_yaw_angle                     1.103386
rectified_left_pitch_angle                   0.881471
rectified_right_roll_angle                   0.164423
rectified_right_yaw_angle                    1.099608
rectified_right_pitch_angle                  0.882569
right_roll_angle                             0.261024
right_yaw_angle                              1.087060
right_pitch_angle                            1.135407
left_right_roll_diff                         0.180036
left_right_yaw_diff                          1.182381
left_right_pitch_diff                        0.939107
left_color_roll_diff        

In [39]:
np.mean(np.abs(results_df["color_roll_angle"].to_numpy()))

0.26000655

In [40]:
mount_roll_angle = np.mean([
	np.mean(results_df["color_roll_angle"].to_numpy()),
	np.mean(results_df["left_roll_angle"].to_numpy()),
	np.mean(results_df["right_roll_angle"].to_numpy())
])

mount_yaw_angle = np.mean([
	np.mean(results_df["color_yaw_angle"].to_numpy()),
	np.mean(results_df["left_yaw_angle"].to_numpy()),
	np.mean(results_df["right_yaw_angle"].to_numpy())
])

mount_pitch_angle = np.mean([
	np.mean(results_df["color_pitch_angle"].to_numpy()),
	np.mean(results_df["left_pitch_angle"].to_numpy()),
	np.mean(results_df["right_pitch_angle"].to_numpy())
])

print(f"mount_roll_angle: {mount_roll_angle}")
print(f"mount_yaw_angle: {mount_yaw_angle}")
print(f"mount_pitch_angle: {mount_pitch_angle}")

results_corrected = results_df.copy()
results_corrected["color_roll_angle"] -= mount_roll_angle
results_corrected["left_roll_angle"] -= mount_roll_angle
results_corrected["right_roll_angle"] -= mount_roll_angle
results_corrected["rectified_right_roll_angle"] -= mount_roll_angle
results_corrected["rectified_left_roll_angle"] -= mount_roll_angle

results_corrected["color_yaw_angle"] -= mount_yaw_angle
results_corrected["left_yaw_angle"] -= mount_yaw_angle
results_corrected["right_yaw_angle"] -= mount_yaw_angle
results_corrected["rectified_right_yaw_angle"] -= mount_yaw_angle
results_corrected["rectified_left_yaw_angle"] -= mount_yaw_angle

results_corrected["color_pitch_angle"] -= mount_pitch_angle
results_corrected["left_pitch_angle"] -= mount_pitch_angle
results_corrected["right_pitch_angle"] -= mount_pitch_angle
results_corrected["rectified_right_pitch_angle"] -= mount_pitch_angle
results_corrected["rectified_left_pitch_angle"] -= mount_pitch_angle



mount_roll_angle: -0.21500177681446075
mount_yaw_angle: 0.47804726733081077
mount_pitch_angle: 0.2211693323440832


In [41]:
results_corrected["color_roll_angle_ok"] = np.abs(results_corrected["color_roll_angle"]) < 0.5
results_corrected["color_yaw_angle_ok"] = np.abs(results_corrected["color_yaw_angle"]) < 1.0
results_corrected["color_pitch_angle_ok"] = np.abs(results_corrected["color_pitch_angle"]) < 1.2

results_corrected["left_roll_angle_ok"] = np.abs(results_corrected["left_roll_angle"]) < 0.5
results_corrected["left_yaw_angle_ok"] = np.abs(results_corrected["left_yaw_angle"]) < 2.0
results_corrected["left_pitch_angle_ok"] = np.abs(results_corrected["left_pitch_angle"]) < 1.5

results_corrected["rectified_right_roll_angle_ok"] = np.abs(results_corrected["rectified_right_roll_angle"]) < 0.5
results_corrected["rectified_right_yaw_angle_ok"] = np.abs(results_corrected["rectified_right_yaw_angle"]) < 2.0
results_corrected["rectified_right_pitch_angle_ok"] = np.abs(results_corrected["rectified_right_pitch_angle"]) < 1.5

results_corrected["rectified_left_roll_angle_ok"] = np.abs(results_corrected["rectified_left_roll_angle"]) < 0.5
results_corrected["rectified_left_yaw_angle_ok"] = np.abs(results_corrected["rectified_left_yaw_angle"]) < 2.0
results_corrected["rectified_left_pitch_angle_ok"] = np.abs(results_corrected["rectified_left_pitch_angle"]) < 1.5

results_corrected["right_roll_angle_ok"] = np.abs(results_corrected["right_roll_angle"]) < 0.5
results_corrected["right_yaw_angle_ok"] = np.abs(results_corrected["right_yaw_angle"]) < 2.0
results_corrected["right_pitch_angle_ok"] = np.abs(results_corrected["right_pitch_angle"]) < 1.5


# results_corrected["left_right_roll_diff_ok"] = results_corrected["left_right_roll_diff"] < 0.3
# results_corrected["left_right_yaw_diff_ok"] = results_corrected["left_right_yaw_diff"] < 0.8
# results_corrected["left_right_pitch_diff_ok"] = results_corrected["left_right_pitch_diff"] < 0.15



In [42]:
results_corrected.transpose().to_excel("results.xlsx")

In [43]:
list(combinations([1,2,3], r=2))

[(1, 2), (1, 3), (2, 3)]